# VacationPy
---

## Starter Code to Import Libraries and Load the Weather and Coordinates Data

In [1]:
# Dependencies and Setup
import hvplot.pandas
import pandas as pd
import requests
import json

# Import API key
from api_keys import geoapify_key

In [2]:
# Load the CSV file created in Part 1 into a Pandas DataFrame
city_data_df = pd.read_csv("output_data/cities.csv")

# Display sample data
city_data_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,adamstown,-25.0660,-130.1015,21.59,70,20,3.49,PN,1701399198
1,1,yalta,44.5000,34.1667,8.17,94,56,0.99,UA,1701399073
2,2,nuuk,64.1835,-51.7216,4.34,75,100,4.63,GL,1701399199
3,3,wailua homesteads,22.0669,-159.3780,24.24,92,100,4.63,US,1701399199
4,4,edinburgh of the seven seas,-37.0676,-12.3116,12.50,72,93,7.45,SH,1701399200


---

### Step 1: Create a map that displays a point for every city in the `city_data_df` DataFrame. The size of the point should be the humidity in each city.

In [3]:
%%capture --no-display

!pip install cartopy
!pip install geoviews
!pip install pyproj

# Configure the map plot
all_city_map = city_data_df.hvplot.points(
    "Lng",
    "Lat",
    geo = True,
    tiles = "OSM",
    frame_width = 800,
    frame_height = 600,
    size = "Humidity",
    alpha = 0.5,
    color = 'Country'
)

# Display the map
all_city_map

:Overlay
   .Tiles.I  :Tiles   [x,y]
   .Points.I :Points   [Lng,Lat]   (Country,Humidity)

### Step 2: Narrow down the `city_data_df` DataFrame to find your ideal weather condition

In [4]:
# Narrow down cities that fit criteria and drop any results with null values

city_temp0 = city_data_df.loc[city_data_df['Max Temp']<27,:]
city_temp = city_temp0.loc[city_temp0['Max Temp']>21,:]
city_wind = city_temp.loc[city_temp['Wind Speed']<4.5,:]
city_cloud = city_wind.loc[city_wind['Cloudiness']==0,:]
city_cloud
# Drop any rows with null values
city_ideal = city_cloud.dropna()
# Display sample data
city_ideal

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
54,54,valle de la pascua,9.2155,-66.0073,22.34,85,0,2.45,VE,1701399127
138,138,vila velha,-20.3297,-40.2925,25.97,84,0,4.47,BR,1701399267
155,155,myitkyina,25.3833,97.4000,22.18,48,0,0.84,MM,1701399276
284,284,sur,22.5667,59.5289,23.81,64,0,2.28,OM,1701399340
348,348,sukhothai,17.2500,99.6667,25.94,83,0,2.06,TH,1701399372
349,349,mar del plata,-38.0023,-57.5575,22.01,68,0,3.60,AR,1701399362
364,364,callao,-12.0667,-77.1500,22.62,77,0,4.12,PE,1701399097
379,379,yanbu,24.0891,38.0637,23.02,83,0,0.00,SA,1701399388
390,390,matoury,4.8483,-52.3317,26.98,78,0,1.54,GF,1701399394
468,468,umm al quwain city,25.5647,55.5552,22.01,67,0,2.57,AE,1701399433


### Step 3: Create a new DataFrame called `hotel_df`.

In [5]:
# Use the Pandas copy function to create DataFrame called hotel_df to store the city, country, coordinates, and humidity
hotel_df = city_ideal.loc[:,['City','Country','Lat','Lng','Humidity']]
# Add an empty column, "Hotel Name," to the DataFrame so you can store the hotel found using the Geoapify API
hotel_df['Hotel Name'] = ''
# Display sample data
hotel_df

,City,Country,Lat,Lng,Humidity,Hotel Name
54,valle de la pascua,VE,9.2155,-66.0073,85,
138,vila velha,BR,-20.3297,-40.2925,84,
155,myitkyina,MM,25.3833,97.4000,48,
284,sur,OM,22.5667,59.5289,64,
348,sukhothai,TH,17.2500,99.6667,83,
349,mar del plata,AR,-38.0023,-57.5575,68,
364,callao,PE,-12.0667,-77.1500,77,
379,yanbu,SA,24.0891,38.0637,83,
390,matoury,GF,4.8483,-52.3317,78,
468,umm al quwain city,AE,25.5647,55.5552,67,


### Step 4: For each city, use the Geoapify API to find the first hotel located within 10,000 metres of your coordinates.

In [6]:
# Set parameters to search for a hotel
radius = 10000
params = {
    "apiKey":geoapify_key,
    "categories":"accommodation.hotel"
}

# Print a message to follow up the hotel search
print("Starting hotel search")

# Iterate through the hotel_df DataFrame
for index, row in hotel_df.iterrows():
    # get latitude, longitude from the DataFrame
    lat = row['Lat']
    lon = row['Lng']
    
    # Add filter and bias parameters with the current city's latitude and longitude to the params dictionary
    params["filter"] = f"circle:{lon},{lat},{radius}"
    params["bias"] = f"proximity:{lon},{lat}"
    
    # Set base URL
    base_url = "https://api.geoapify.com/v2/places"

    # Make and API request using the params dictionaty
    name_address = requests.get(base_url, params=params)
    
    # Convert the API response to JSON format
    name_address = name_address.json()
    
    # Grab the first hotel from the results and store the name in the hotel_df DataFrame
    try:
        hotel_df.loc[index, "Hotel Name"] = name_address["features"][0]["properties"]["name"]
    except (KeyError, IndexError):
        # If no hotel is found, set the hotel name as "No hotel found".
        hotel_df.loc[index, "Hotel Name"] = "No hotel found"
        
    # Log the search results
    print(f"{hotel_df.loc[index, 'City']} - nearest hotel: {hotel_df.loc[index, 'Hotel Name']}")

# Display sample data
hotel_df

Starting hotel search
valle de la pascua - nearest hotel: Palace Hotel
vila velha - nearest hotel: Hotel Prainha
myitkyina - nearest hotel: New Light Hotel
sur - nearest hotel: Sur Hotel
sukhothai - nearest hotel: No hotel found
mar del plata - nearest hotel: Gran hotel Mónaco
callao - nearest hotel: Casa Ronald
yanbu - nearest hotel: No hotel found
matoury - nearest hotel: La Marmotte
umm al quwain city - nearest hotel: Palma Beach Resorts
pamandzi - nearest hotel: Gite Le Madina


,City,Country,Lat,Lng,Humidity,Hotel Name
54,valle de la pascua,VE,9.2155,-66.0073,85,Palace Hotel
138,vila velha,BR,-20.3297,-40.2925,84,Hotel Prainha
155,myitkyina,MM,25.3833,97.4000,48,New Light Hotel
284,sur,OM,22.5667,59.5289,64,Sur Hotel
348,sukhothai,TH,17.2500,99.6667,83,No hotel found
349,mar del plata,AR,-38.0023,-57.5575,68,Gran hotel Mónaco
364,callao,PE,-12.0667,-77.1500,77,Casa Ronald
379,yanbu,SA,24.0891,38.0637,83,No hotel found
390,matoury,GF,4.8483,-52.3317,78,La Marmotte
468,umm al quwain city,AE,25.5647,55.5552,67,Palma Beach Resorts


### Step 5: Add the hotel name and the country as additional information in the hover message for each city in the map.

In [7]:
%%capture --no-display

# Configure the map plot
hotel_map = hotel_df.hvplot.points(
    "Lng",
    "Lat",
    geo = True,
    tiles = "OSM",
    frame_width = 800,
    frame_height = 600,
    size = "Humidity",
    alpha =0.8,
    color = 'City',
    hover_cols = ["Hotel Name", "Country"]
)

# Display the map
hotel_map

:Overlay
   .Tiles.I  :Tiles   [x,y]
   .Points.I :Points   [Lng,Lat]   (City,Humidity,Hotel Name,Country)